## Random forest model

In [2]:
%sh
pip install pydotplus
sudo apt-get install -y graphviz




Requirement already satisfied: pydotplus in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (2.0.2)
Requirement already satisfied: pyparsing>=2.0.1 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from pydotplus) (2.4.6)
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
 fonts-liberation libann0 libcdt5 libcgraph6 libgts-0.7-5 libgts-bin libgvc6
 libgvpr2 liblab-gamut1 libpathplan4
Suggested packages:
 graphviz-doc
The following NEW packages will be installed:
 fonts-liberation graphviz libann0 libcdt5 libcgraph6 libgts-0.7-5 libgts-bin
 libgvc6 libgvpr2 liblab-gamut1 libpathplan4
0 upgraded, 11 newly installed, 0 to remove and 1 not upgraded.
Need to get 2,670 kB of archives.
After this operation, 11.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 fonts-liberation all 1:1.07.4-7~18.04.1 [822 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libann0 amd64 1.1.2+doc-6 [24.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libcdt5 amd64 2.40.1-2 [19.6 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libcgraph6 amd64 2.40.1-2 [40.8 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgts-0.7-5 amd64 0.7.6+darcs121130-4 [150 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libpathplan4 amd64 2.40.1-2 [22.6 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgvc6 amd64 2.40.1-2 [601 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgvpr2 amd64 2.40.1-2 [169 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic/universe amd64 liblab-gamut1 amd64 2.40.1-2 [178 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic/universe amd64 graphviz amd64 2.40.1-2 [601 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgts-bin amd64 0.7.6+darcs121130-4 [41.3 kB]
debconf: delaying package configuration, since apt-utils is not installed
Fetched 2,670 kB in 2s (1,456 kB/s)
Selecting previously unselected package fonts-liberation.
(Reading database ... 
(Reading database ... 5%
(Reading database ... 10%
(Reading database ... 15%
(Reading database ... 20%
(Reading database ... 25%
(Reading database ... 30%
(Reading database ... 35%
(Reading database ... 40%
(Reading database ... 45%
(Reading database ... 50%
(Reading database ... 55%
(Reading database ... 60%
(Reading database ... 65%
(Reading database ... 70%
(Reading database ... 75%
(Reading database ... 80%
(Reading database ... 85%
(Reading database ... 90%
(Reading database ... 95%
(Reading database ... 100%
(Reading database ... 92546 files and directories currently installed.)
Preparing to unpack .../00-fonts-liberation_1%3a1.07.4-7~18.04.1_all.deb ...
Unpacking fonts-liberation (1:1.07.4-7~18.04.1) ...
Selecting previously unselected package libann0.
Preparing to unpack .../01-libann0_1.1.2+doc-6_amd64.deb ...
Unpacking libann0 (1.1.2+doc-6) ...
Selecting previously unselected package libcdt5.
Preparing to unpack .../02-libcdt5_2.40.1-2_amd64.deb ...
Unpacking libcdt5 (2.40.1-2) ...
Selecting previously unselected package libcgraph6.
Preparing to unpack .../03-libcgraph6_2.40.1-2_amd64.deb ...
Unpacking libcgraph6 (2.40.1-2) ...
Selecting previously unselected package libgts-0.7-5:amd64.
Preparing to unpack .../04-libgts-0.7-5_0.7.6+darcs121130-4_amd64.deb ...
Unpacking libgts-0.7-5:amd64 (0.7.6+darcs121130-4) ...
Selecting previously unselected package libpathplan4.
Preparing to unpack .../05-libpathplan4_2.40.1-2_amd64.deb ...
Unpacking libpathplan4 (2.40.1-2) ...
Selecting previously unselected package libgvc6.
Preparing to unpack .../06-libgvc6_2.40.1-2_amd64.deb ...
Unpacking libgvc6 (2.40.1-2) ...
Selecting previously unselected package libgvpr2.
Preparing to unpack .../07-libgvpr2_2.40.1-2_amd64.deb ...
Unpacking libgvpr2 (2.40.1-2) ...
Selecting previously unselected package liblab-ga

In [3]:
dbutils.library.restartPython()

In [4]:

# Import libraries

from sklearn import tree

from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression,BayesianRidge,Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from statsmodels.api import OLS
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import numpy as np
import pandas as pd
import pydotplus
from re import sub
import scipy.stats as sci
import seaborn as sns
 
# Set visualization prefrences 
sns.set(font_scale=1.5, style="darkgrid")
pd.set_option('display.max_columns', None)

/databricks/python/lib/python3.7/site-packages/patsy/constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
 from collections import Mapping

In [5]:
df = spark.read.parquet('/databricks/driver/spark_merged_modeling.parquet')
#df.show(10)
df= df.toPandas()

In [6]:
non_features = ['county','state','POP_ESTIMATE_2018','total_state_pop', 
                'pop_fraction']

to_be_normalzied_features = ['cases','deaths','All_Specialties_AAMC',
                             'Total_nurse_practitioners_2019',
                             'Total_physician_assistants_2019',
                             'Total_Hospitals_2019',
                             'Total_Primary_Care_Physicians_2019',
                             'Surgery_specialists_2019',
                             'Emergency_Medicine_specialists_2019',
                             'Total_Specialist_Physicians_2019',
                             'ICU_Beds',
                             ]



in_percentage = ['Percentage_of_Active_Physicians_Who_Are_Female_2018_AAMC',
                 'Percentage_of_Active_Physicians_Who_Are_Intertiol_Medical_Graduates_IMGs_2018_AAMC',
                 'Percentage_of_Active_Physicians_Who_Are_Age_60_or_Older_2018_AAMC',
                 'Percentage_Change_in_Student_Enrollment_at_MD_and_DO_Schools_2008_2018_AAMC',
                 'Percentage_of_MD_Students_Matriculating_In_State_AY_2018_2019_AAMC',
                 'Percentage_of_Residents_in_ACGME_Programs_Who_Are_IMGs_as_of_December_31_2018_AAMC',
                 'Percent_Change_in_Residents_and_Fellows_in_ACGME_Accredited_Programs_2008_2018_AAMC',
              'Percentage_of_Physicians_Retained_in_State_from_Undergraduate_Medical_Education_UME_2018_AAMC',
                 ]

In [7]:
X = df[df.columns[~df.columns.isin(non_features)]].drop(['cases','deaths'], axis=1) # Exclude deaths & cases from X
y = df['deaths']

In [8]:
# Split into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, df['deaths'], test_size=0.25, random_state=10)

print('Features size for train(X,y) and test(X_test):')
print('X_train', X_train.shape, 'y_train', y_test.shape, 'X_test', X_test.shape)




Features size for train(X,y) and test(X_test):
X_train (2190, 45) y_train (730,) X_test (730, 45)

In [9]:
#Modeling
#Add lock-down length as an additional feature

# File location and type
file_location = "/FileStore/tables/States_Lockdown_Lengh.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

## import States_Lock-down_Lengh  file 
# The applied options are for CSV files. For other file types, these will be ignored.
States_Lockdown_Lengh = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)
display(States_Lockdown_Lengh)



state,Lock_down_length
Alabama,26
Alaska,41
Arizona,46
Arkansas,#N/A
California,#N/A
Colorado,32
Connecticut,58
Delaware,52
Florida,41
Georgia,27


In [10]:
def regression_fit(model, X_train, y_train, X_test, y_test):
    '''Fit and evaluate a regression model'''
    # Fit model
    model.fit(X_train, y_train)
    
    # Predict test y values
    y_prediction = model.predict(X_test)

    # Calculate root-mean-square error (RMSE), indicative of diffrences between predicted and obseverd values
    RMSE = np.sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction))

    # Calcualte R square, indicative of the percentage of variance of the predicted varialbe explained by the model
    R2 = r2_score(y_test,y_prediction)

    return (model, RMSE,R2)

In [11]:
# Choose training variable (based on gini importance)
train_vars = ['ICU_Beds',
'Adult_obesity_percentage',
'Quality_of_Life_rank',
'Excessive_drinking_percentage',
'Health_Behaviors_rank',
'Population_per_sq_mile',
'Length_of_Life_rank',
'Clinical_Care_rank',
'Adult_smoking_percentage',
'Total_Specialist_Physicians_2019',
'Physical_Environment_rank'
]

In [12]:
# Set regresssion models for testing
reg_models = list()

reg_models = [
    ('Linear regression',LinearRegression()),
    ('Bayesian Ridge regression', BayesianRidge()),
    ('Decision tree',DecisionTreeRegressor(max_depth=1500,random_state=0)),
    ('Random forest',RandomForestRegressor(max_depth=1500,random_state=0)),
    ('Gradient boosting',GradientBoostingRegressor(random_state=0)),
    ('Linear regression with Ridge regularization', Ridge()),
    ('Linear regression with Lasso regularization (L1 penalty)',Lasso(random_state=0)),
    ('Linear regression with ElasticNet regularization',ElasticNet(random_state=0))
     ]

In [13]:
# Train models and get test results.
results = pd.DataFrame(columns = ['Model','RMSE','R-squared'])
models = []

for i,model in enumerate(reg_models):
    trained_model, RMSE, R2 = regression_fit(model[1], X_train[train_vars], y_train, X_test[train_vars], y_test)
    results.loc[i] = [model[0],RMSE,R2]
    models.append(trained_model)

In [14]:
results.sort_values(by='R-squared', ascending=False)

,Model,RMSE,R-squared
4,Gradient boosting,0.154143,0.329383
3,Random forest,0.158728,0.288896
5,Linear regression with Ridge regularization,0.174189,0.143611
0,Linear regression,0.174192,0.143588
1,Bayesian Ridge regression,0.174346,0.142074
7,Linear regression with ElasticNet regularization,0.177559,0.110162
6,Linear regression with Lasso regularization (L...,0.177599,0.109761
2,Decision tree,0.236898,-0.583988


In [15]:
mlp = MLPRegressor(hidden_layer_sizes=(5,5,5), activation='logistic', 
                   learning_rate='adaptive', solver='lbfgs', max_iter=2000, 
                   learning_rate_init=.001, random_state=10)

mlp.fit(X_train[train_vars],y_train)

predict_train = mlp.predict(X_train[train_vars])
predict_test = mlp.predict(X_test[train_vars])

In [16]:
# Calculate root-mean-square error (RMSE), indicative of diffrences between predicted and obseverd values
RMSE = np.sqrt(mean_squared_error(y_true = y_test, y_pred = predict_test))

# Calcualte R square, indicative of the percentage of variance of the predicted varialbe explained by the model
R2 = r2_score(y_test,predict_test)

print(RMSE,R2)
results.loc[i+1] = ['Multilayer Perceptron', RMSE, R2]


0.18297114630893294 0.05508528242483546

In [17]:
# Train a neural netowrk (MLP)

mlp = MLPRegressor(hidden_layer_sizes=(5,5,5), activation='logistic', 
                   learning_rate='adaptive', solver='lbfgs', max_iter=2000, 
                   learning_rate_init=.001, random_state=10)

mlp.fit(X_train[train_vars],y_train)

predict_train = mlp.predict(X_train[train_vars])
predict_test = mlp.predict(X_test[train_vars])

In [18]:
# Calculate root-mean-square error (RMSE), indicative of diffrences between predicted and obseverd values
RMSE = np.sqrt(mean_squared_error(y_true = y_test, y_pred = predict_test))

# Calcualte R square, indicative of the percentage of variance of the predicted varialbe explained by the model
R2 = r2_score(y_test,predict_test)

print(RMSE,R2)
results.loc[i+1] = ['Multilayer Perceptron', RMSE, R2]

0.18297114630893294 0.05508528242483546

In [19]:
# Plot an example tree from the Gradient boosting model
from IPython.display import Image
%matplotlib inline
from IPython.core.display import HTML 
from IPython.display import display
#from pyspark.ml.image import ImageSchema
sub_tree =  models[4].estimators_[10, 0]
dot_data = tree.export_graphviz(
    sub_tree,
    out_file=None, filled=True,
    rounded=True,  
    special_characters=True,
    proportion=True,
    feature_names=X_train[train_vars].columns
)

graph = pydotplus.graph_from_dot_data(dot_data)  
#Image(graph.create_png())
Image(graph.create_png()).show()

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-2002980874548145> in <module> 
 17 graph = pydotplus . graph_from_dot_data ( dot_data ) 
 18 #Image(graph.create_png()) 
 ---> 19 Image ( graph . create_png ( ) ) . show ( ) 

 AttributeError : 'Image' object has no attribute 'show'

In [20]:
im = Image(graph.create_png()).visualize()
open('output.png', 'wb').write(im.data)

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-2002980874548146> in <module> 
 ----> 1 im = Image ( graph . create_png ( ) ) . visualize ( ) 
 2 open ( 'output.png' , 'wb' ) . write ( im . data ) 

 AttributeError : 'Image' object has no attribute 'visualize'